[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/ik-nlp-tutorials/blob/main/notebooks/W3E_BPE_Transduction.ipynb)

In [34]:
# Run in Colab to install local packages
# !pip install spacy transformers sentencepiece tokenizers datasets simplet5
# !python -m spacy download en_core_web_sm

# Training a BPE tokenizer and a Lexicon-based Transduction Model

*This exercise follows the explanation of using BPE tokenization as explained on Huggingface [Build a Tokenizer from Scratch](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html#build-a-tokenizer-from-scratch). Adapted from a notebook by Wietse de Vries*

The [Tokenizers](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html) library by Huggingface provides implementations of today’s most used tokenizers (especially subword-based ones) that is both easy to use and blazing fast (Rust-compiled code!).

You will start by exploring the impact of different vocabulary sizes on a subword tokenizer using the Tokenizers library, and how these can be imported and used with spaCy. Finally, you will be asked to train a small transformer model to perform transduction from feminine to masculine words.

Exercise 1 is mandatory and will be part of your graded midterm portfolio. Exercise 2 is optional, but we highly recommend you to complete it, especially if you're interested in the "Modern Neural Networks meet Linguistic Theory" final project.

## Exercise 1: Byte Pair Encoding with Huggingface Tokenizers

In the following exercise, we will use a byte-pair encoding (BPE) tokenizer (see Jurafsky & Martin Sec. 2.4.3 and [Sennich et al, 2015](https://aclanthology.org/P16-1162/) to create a vocabulary of frequent words and subwords, allowing us to handle less frequent words.

### Setup

The following code loads a BPE tokenizer and trainer, tells the system to use whitespace as a separator and defines `[UNK]` as a special token intended to handle unknown words.

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]"], vocab_size=20000) 

### Corpus

The tokenizer creates a dictionary by concatenating characters and substrings into longer strings (possibly full words) based on frequency. So we need a corpus to learn what the most frequent words and substrings are. 

[Wikitext-103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) is a dump of the (English) Wikipedia. You can use the `train_from_iterator` method to train from the data in memory, which can be done using the `wikitext` corpus in the Huggingface Datasets library. Alternatively, you can download using wget, or directly from the webpage:

```shell
wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
unzip wikitext-103-raw-v1.zip
```

The unzipped data is 500 MB. Note that the file extension for the data-files is .raw but the data is just a (unicode) text file. Because this confuses (Ubuntu) Linux, files were renamed to .raw.txt. If you maintain the original .raw filenames, adapt the path below accordingly.

### Run the trainer

The command below trains the tokenizer on the data:

In [2]:
# UNCOMMENT AND ADAPT PATH TO TRAIN ON MANUALLY DOWNLOADED DATASETS
# data = [f'wikitext-103-raw/wiki.{split}.raw.txt' for split in ['train','test','valid']]
# tokenizer.train(trainer, data)

import datasets
dataset = datasets.load_dataset(
    "wikitext", "wikitext-103-raw-v1", split="train+test+validation"
)

# Build a generator to iterate over the dataset
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(dataset))

### Test the tokenizer

Now that we have created a vocabulary, we can use it to tokenize a string into words and subtokens (for infrequent words).

The example shows that most of the words are included in the vocabulary created by training on Wikipedia text, but that the acronym *UG*, the name *Hanze*, and the word *Applied*, *jointly* and *initiating* are segmented into subword strings. This suggests that these words were not seen during training, or very infrequently. (*UG* occurs 5 times in the training data and *Applied* over 200 times,  also note that the encoding is case-sensitive.). 

Try a few other examples to get a feeling for the lexical coverage of the tokenizer. 

In [37]:
def show_tokens(text):
    output = tokenizer.encode(text)
    print(f"Tokens: {output.tokens}")
    number_of_words = len(tokenizer.pre_tokenizer.pre_tokenize_str(text))
    number_of_segments = len(output.tokens)
    print(f"{number_of_words} words and {number_of_segments} segments")

example = "The UG and the Hanze University of Applied Sciences are jointly initiating a pilot rapid testing centre, which will start on 18 January."
show_tokens(example)

Tokens: ['The', 'U', 'G', 'and', 'the', 'Han', 'ze', 'University', 'of', 'Ap', 'pl', 'ied', 'Sciences', 'are', 'joint', 'ly', 'initi', 'ating', 'a', 'pilot', 'rapid', 'testing', 'centre', ',', 'which', 'will', 'start', 'on', '18', 'January', '.']
25 words and 31 segments


### Your Turn: Experiment with Vocabulary Size

The training data contains 103 M tokens and has a vocabulary size of 267,000 unique types. The default setting for the trainer is to create a dictionary of max 30,000 words. This means that a fair amount of compression takes place. Even more compression can be achieved by setting the vocab_size to a smaller value. 

1. Choose an example text consisting of at least 100 words. You may want to ensure that it contains some rare words or tokens. 

2. Experiment with various settings for vocab_size.

3. Count the number of words in the example, and the number of segments created by the BPE-tokenizer. Note that if the number segments goes up, more words are segmented into subwords. 

4. What is the vocabulary size where the number of segments is approx. 150% of the number of words? 

5. For this setting, what was the longest word in your example text that was not segmented? 

In [38]:

# test text source: https://en.wikipedia.org/wiki/Saab_JAS_39_Gripen  (154 words according to me, the tokenizer says 188 words)
test_text = "The Saab JAS 39 Gripen is a light single-engine supersonic multirole fighter aircraft manufactured by the Swedish aerospace and defence company Saab AB. The Gripen has a delta wing and canard configuration with relaxed stability design and fly-by-wire flight controls. Later aircraft are fully NATO interoperable. As of 2020, more than 271 Gripens of all models, A–F, have been delivered. In 1979, the Swedish government began development studies for \"an aircraft for fighter, attack, and reconnaissance\" (ett jakt-, attack- och spaningsflygplan, hence \"JAS\") to replace the Saab 35 Draken and 37 Viggen in the Swedish Air Force. A new design from Saab was selected and developed as the JAS 39. The first flight took place in 1988, with delivery of the first serial production airplane in 1993. It entered service with the Swedish Air Force in 1996. Upgraded variants, featuring more advanced avionics and adaptations for longer mission times, began entering service in 2003."


# loop over different vocab sizes and compare the number of segments with the number of words
vocab_sizes = [1000, 5000, 6500, 8000, 10000, 20000, 30000, 100000]
vocab_sizes.reverse()
for vocab_size in vocab_sizes:
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = BpeTrainer(special_tokens=["[UNK]"],vocab_size=vocab_size)
    tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(dataset))
    output = tokenizer.encode(test_text)
    number_of_words = len(tokenizer.pre_tokenizer.pre_tokenize_str(test_text))
    number_of_segments = len(output.tokens)
    print(f"Vocab size: {vocab_size}, {number_of_words} words and {number_of_segments} segments")
    
    # find out when vocab size is 150% of the number of segments
    if number_of_segments > 1.5 * number_of_words and number_of_segments < (1.5 * number_of_words * 1.1):
        print(f"The vocab size where the number of segments is approx. 150% of the number of words is {vocab_size}")
    
        # find the longest word that was not segmented (question 5)
        words = tokenizer.pre_tokenizer.pre_tokenize_str(test_text)
        for word in words:
            if len(word) == 1:
                print(f"q5: Longest word that was not segmented: {word} with vocab size {vocab_size}")





Vocab size: 100000, 188 words and 201 segments

Vocab size: 30000, 188 words and 219 segments

Vocab size: 20000, 188 words and 227 segments

Vocab size: 10000, 188 words and 268 segments

Vocab size: 8000, 188 words and 297 segments
The vocab size where the number of segments is approx. 150% of the number of words is 8000

Vocab size: 6500, 188 words and 330 segments


Vocab size: 5000, 188 words and 822 segments


Vocab size: 1000, 188 words and 822 segments


### Loading the BPE Tokenizer into spaCy

Now that you experimented with the creation of many tokenizers using Huggingface Tokenizers, you might want to move them to a more familiar environment. The following class lets you load a Huggingface Tokenizer into spaCy: the `get_words_spaces` function is used to preserve the whitespaces before tokens that are not word pieces.

### Your Turn: Fill in the missing code

Your task is to complete the `__call__` method of the `BPETokenizer` class to go from text to spaCy `Docs`, and finally to print the tokenized text.

In [39]:
# for same reason after running above cell the tokenizer is not working anymore, so I have to redo the setup

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]"], vocab_size=20000) 


dataset = datasets.load_dataset(
    "wikitext", "wikitext-103-raw-v1", split="train+test+validation"
)

# Build a generator to iterate over the dataset
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(dataset))

In [33]:
from spacy.tokens import Doc
from spacy.vocab import Vocab
import spacy

class BPETokenizer:
    def __init__(self, tokenizer, vocab):
        self.tokenizer = tokenizer
        self.vocab = vocab
    
    def get_words_spaces(self, tokens):
        words = []
        spaces = []
        for i, (text, (_, end)) in enumerate(
            zip(tokens.tokens, tokens.offsets)
        ):
            words.append(text)
            if i < len(tokens.tokens) - 1:
                # If next start != current end we assume a 
                # space in between
                next_start, _ = tokens.offsets[i + 1]
                spaces.append(next_start > end)
            else:
                spaces.append(True)
        return words, spaces

    def __call__(self, text):
        # Encode the texts to obtain tokens
        tokens = self.tokenizer.encode(text)
        # Use get_words_spaces to obtain the words and spaces
        words, spaces = self.get_words_spaces(tokens)
        return Doc(self.vocab, words=words, spaces=spaces)

nlp = spacy.blank("en")
nlp.vocab = Vocab(strings=[
    tok for tok in tokenizer.get_vocab().keys()
])
nlp.tokenizer = BPETokenizer(tokenizer, nlp.vocab)

text = "Jeff Bezos is a billionaire who became famous after the Dutch bridge controversy."
# Convert the text in a list of tokens and print them
doc = nlp(text)
print([token.text for token in doc])

['Jeff', 'Be', 'z', 'os', 'is', 'a', 'billion', 'aire', 'who', 'became', 'famous', 'after', 'the', 'Dutch', 'bridge', 'controversy', '.']


## (Optional) Exercise 2: Lexicon-based Transduction System

In this exercise you will build a rule-based tool to transduce a given input text **from masculine to feminine**. You are provided with a list of pairs including feminine words and their masculine counterparts. To create a rule based transducer, the following components will be needed:

1. Extract a subset of sentences from the `wikitext-103-raw-v1` containing masculine words (words from the list, gendered pronouns (e.g. he/his/him)). **Tip**: you can try to use the spaCy lemmas annotations to avoid removing inflected forms of words.

Fill the `is_masculine` function so that only sentences containing masculine words are preserved.

In [3]:
import re
import datasets

gender_lexicon = [
    ("Brother", "Sister"),
    ("Drake", "Duck"),
    ("Father", "Mother"),
    ("Gentleman", "Lady"),
    ("Husband", "Wife"),
    ("Man", "Woman"),
    ("Nephew", "Niece"),
    ("Son", "Daughter"),
    ("Wizard", "Witch"),
    ("Boy", "Girl"),
    ("Bull", "Cow"),
    ("Cock", "Hen"),
    ("Dog", "Bitch"),
    ("Drone", "Bee"),
    ("Gander", "Goose"),
    ("Horse", "Mare"),
    ("King", "Queen"),
    ("Monk", "Nun"),
    ("Sir", "Madam"),
    ("Stag", "Hind"),
    ("Stallion", "Mare"),
    ("Tutor", "Governess"),
    ("Drone", "Bee"),
    ("Brother-in-law", "Sister-in-law"),
    ("Son-in-law", "Daughter-in-law"),
    ("Maternal-uncle", "Maternal-aunt"),
    ("Step-son", "Step-daughter"),
    ("Hostess", "Steward"),
    ("Widow", "Widower"),
    ("author", "authoress"),
    ("count", "countess"),
    ("heir", "heiress"),
    ("manager", "manageress"),
    ("patron", "patroness"),
    ("priest", "priestess"),
    ("baron", "baroness"),
    ("giant", "giantess"),
    ("host", "hostess"),
    ("lion", "lioness"),
    ("mayor", "mayoress"),
    ("poet", "poetess"),
    ("shepherd", "shepherdess"),
    ("actor", "actress"),
    ("conductor", "conductress"),
    ("hunter", "huntress"),
    ("prince", "princess"),
    ("traitor", "traitress"),
    ("master", "mistress"),
    ("benefactor", "benefactress"),
    ("founder", "foundress"),
    ("instructor", "instructress"),
    ("emperor", "empress"),
    ("tiger", "tigress"),
    ("waiter", "waitress"),
    ("murderer", "murderess"),
    ("hero", "heroine"),
    ("fox", "vixen"),
    ("sultan", "sultana"),
    ("grandfather", "grandmother"),
    ("manservant", "maidservant"),
    ("milkman", "milkwoman"),
    ("salesman", "saleswoman"),
    ("great-uncle", "great-aunt"),
    ("landlord", "landlady"),
    ("he", "she"),
    ("him", "her"),
    ("his", "her")
]

def is_masculine(text):
    # (use '|'.join(...) to join them in the regex)
    masculine_words = []
    
    for pair in gender_lexicon:
        masculine_words.append(pair[0])
    
    regex = r'\b(' + '|'.join(masculine_words) + r')\b'
    return bool(re.search(regex, text, re.IGNORECASE))


dataset = datasets.load_dataset(
    "wikitext", "wikitext-103-raw-v1", split="train+test+validation"
)

# We consider only the first 200 characters to avoid long paragraphs
filtered_dataset = dataset.filter(lambda x: is_masculine(x["text"][:200]))
filtered_dataset = filtered_dataset.map(lambda x: {"text": x["text"][:200]})
filtered_dataset

Dataset({
    features: ['text'],
    num_rows: 207019
})

In [45]:
# print some examples from the filtered dataset
for i in range(5):
    print(filtered_dataset[i]["text"])

 The music was composed by Hitoshi Sakimoto , who had also worked on the previous Valkyria Chronicles games . When he originally heard about the project , he thought it would be a light tone similar t
 Perhaps because Abraham Lincoln had not yet been inaugurated as President , Captain Totten received no instructions from his superiors and was forced to withdraw his troops . He agreed to surrender t
 The military force at Little Rock under Dunnington 's command consisted of four officers : himself , Major John B. Lockman , Captain C.C. Green , and 2nd Lt. W.W. Murphy . In addition to these , he h
 Lt. Col. Dunnington continued to build up his works at Little Rock until November 1862 , when Captain Sanford C. Faulkner ( composer of The Arkansas Traveler ) was placed in charge of the Arsenal . D
 Barker was born the second daughter and youngest child of Walter Barker , a partner in a seed supply company and an amateur artist , and his wife Mary Eleanor ( Oswald ) Barker on 28 June 1895 at

2. Create a `feminize` function that takes a sententence from the the filtered dataset and returns a feminized version of it, based on lexical pairs. Use it to create a new field "feminine_text" in the dataset.

In [4]:
def feminize(text):
    """Returns a feminized version of text"""
    feminized_text = text
    for m, f in gender_lexicon:
        match_regex = r'\b' + re.escape(m) + r'\b' 
        feminized_text = re.sub(match_regex, f, feminized_text, flags=re.IGNORECASE)
    return feminized_text

# TODO: Use filtered_dataset.map to add a feminized version of the text column
feminized_dataset = filtered_dataset.map(lambda x: {"feminine_text": feminize(x["text"])})
feminized_dataset

Dataset({
    features: ['text', 'feminine_text'],
    num_rows: 207019
})

In [5]:
# print some examples from the feminized dataset
for i in range(5):
    print(feminized_dataset[i]["feminine_text"])

 The music was composed by Hitoshi Sakimoto , who had also worked on the previous Valkyria Chronicles games . When she originally heard about the project , she thought it would be a light tone similar t
 Perhaps because Abraham Lincoln had not yet been inaugurated as President , Captain Totten received no instructions from her superiors and was forced to withdraw her troops . she agreed to surrender t
 The military force at Little Rock under Dunnington 's command consisted of four officers : himself , Major John B. Lockman , Captain C.C. Green , and 2nd Lt. W.W. Murphy . In addition to these , she h
 Lt. Col. Dunnington continued to build up her works at Little Rock until November 1862 , when Captain Sanford C. Faulkner ( composer of The Arkansas Traveler ) was placed in charge of the Arsenal . D
 Barker was born the second daughter and youngest child of Walter Barker , a partner in a seed supply company and an amateur artist , and her wife Mary Eleanor ( Oswald ) Barker on 28 June 189

3. Rename the `text` field to `source_text` and the `feminine_text` field to `target_text` (this is needed for `SimpleT5` to work properly). Transform the dataset to Pandas DataFrame format and use the following code to train a simple neural transduction model.

*(More info on the [T5 model](https://huggingface.co/t5-small) and the [SimpleT5](https://github.com/Shivanandroy/simpleT5) library)*

In [7]:
# clear gpu memory
import torch
torch.cuda.empty_cache()
# clear all venv gpu memory


In [8]:

from simplet5 import SimpleT5
from sklearn.model_selection import train_test_split
import pandas as pd

# TODO: Convert the Huggingface Dataset in a Pandas dataframe and split it in training
# and evaluation sets (you decide the sizes based on your computational resources)
df = pd.DataFrame(feminized_dataset)
df.rename(columns={"text": "source_text", "feminine_text": "target_text"}, inplace=True)
train_df, eval_df = train_test_split(df, test_size=0.2)


model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")
model.train(
    train_df=train_df,
    eval_df=eval_df, 
    source_max_token_len=128, 
    target_max_token_len=128, 
    batch_size=40, max_epochs=3, use_gpu=torch.cuda.is_available()
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/matisse/ik-nlp-tutorials/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Global seed set to 42
/home/matisse/ik-nlp-tutorials/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the enviro

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the enviro

Validating: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
# save the model to current directory named "t5-modeltje"
model.save_model("model")

AttributeError: 'SimpleT5' object has no attribute 'save_model'

4. Conclude by testing the model on a few examples of your choice

In [13]:
model.load_model("t5", "outputs/simplet5-epoch-2-train-loss-0.0009-val-loss-0.0002", use_gpu=torch.cuda.is_available())

text_to_feminize = "my brother thought that his uncle was a duke"
model.predict(text_to_feminize)

['my Sister thought that her uncle was a duke']